In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

In [ ]:
from azureml.core import Workspace, Dataset

subscription_id = '###subscription-id'
resource_group = '###resource-group'
workspace_name = '###ml-workspace'

ws = Workspace(subscription_id, resource_group, workspace_name)

exp = ws.experiments['###experiment-name']

e = list(exp.get_runs())[0]
print(e)

In [ ]:
aml_run_details = e.get_details()
aml_run_details

In [ ]:
#By default gets the best performing model
aml_run = list(ws.experiments['##experiment-name'].get_runs())[0]
best_run, fitted_model = aml_run.get_output()

In [ ]:
import pandas as pd

out=pd.DataFrame()

# Retrieve accuracy metrics for automl experiment
for child in (aml_run.get_children()) :
    if(('run_algorithm' in child.get_details()['properties']) and ('accuracy' in child.get_metrics())):
        model_output = {
            'iteration': [int(child.get_details()['properties']['iteration'])],
            'run_id' : child.id,
            'algo': [child.get_details()['properties']['run_algorithm']],
            'accuracy': [child.get_metrics()['accuracy']]
        }
        #print(model_output)
        out = pd.concat([out,pd.DataFrame.from_dict(model_output)])

out = out.reset_index()
out
    

In [ ]:
import seaborn as sns
import matplotlib as plt

#plot accuracy acreoss auto-ml runs
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.lineplot(data = out,x='iteration', y='accuracy', hue='algo', style='algo', markers=True)

In [ ]:
# retrieve the most accurate XGBoostClassifier model
best_xgb = (out[out['algo'] == 'XGBoostClassifier'].sort_values(by='accuracy', ascending=False)).head(1)

best_run, fitted_xgbmodel = aml_run.get_output(int(best_xgb['iteration']))

# Print model details 

In [ ]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_xgbmodel)